## 2025년 천안시 데이터 분석 공모전


✅주제: 천안시 산업재해와 지역 사회경제·환경·복지 요인(고용, 기상, 소비, 주거, 건강 등) 간의 관계를 시계열적으로 분석하고, <br>
<span style='margin:30px;'>이를 통해 산업안전 및 취약계층 보호를 위한 예측 모델<br><br>
✅자료출처: MDIS, 통계청<br><br>
✅분석순서
<ol> 
    <li>데이터 로드</li>
    <li>데이터 EDA</li>
    <li>변수 선정</li>
    <li>ARIMA, 머신러닝 등으로 시계열 예측하기</li>
</ol>


In [1]:
# 패키지 로드
import glob 
import pandas as pd

### 1. 입력 데이터 로드

#### 👉 주관적 건강 수준 (입력데이터)
- 출처: 보건복지부
- 대상범위: 전국, 임금근로자
- 기간: 2019~2023년

In [55]:
# 데이터 로드 
health = pd.read_csv('1. 기초자료\보건복지부_주관적_건강_수준.csv')

# 칼럼 정리 
health.loc[0] = health.apply(lambda x: 5 if x.loc[0]=="매우 좋다" else 4 if x.loc[0]=="좋은 편이다"  # 각 항목 점수로 매핑
                             else 3 if x.loc[0]=="보통이다"else 2 if x.loc[0]=="나쁜 편이다" else 1 if x.loc[0]=='매우 나쁘다' else x.iloc[0])

health = health[health['특성별(2)'].isin(['특성별(2)','임금근로자'])].copy() # 임금근로자만 필터
health = health.T.iloc[2:] # 행열전환
health = health.reset_index() # 인덱스해제
health.columns = ['날짜','만족도','응답비율']
health['날짜'] = health['날짜'].str.slice(0,4) # 날짜 정리
health.head(3)

,날짜,만족도,응답비율
0,2019,5,16.4
1,2019,4,64.9
2,2019,3,16.4


#### 👉 삶에 대한 만족감 (입력데이터)
- 출처: 충남
- 대상범위: 충남(천안시)
- 기간: 2019~2023년

In [85]:
# 루트위치
root = '1. 기초자료/'
# 파일명
file_name = [
            '삶에_대한_만족감_2019.csv',
            '삶에_대한_만족감_2020.csv',
            '삶에_대한_만족감_2021.csv',
            '삶에_대한_만족감_2022.csv',
            '삶에_대한_만족감_2023.csv',
            ]
# 파일정리 + 파일 취합
def make_file(file_name): 
    try:
        df=pd.read_csv(root+file_name)
    except:
        df=pd.read_csv(root+file_name,encoding='cp949')
    try:
        df=df[df['응답자특성별(2)'].isin(['천안시','응답자특성별(2)'])].T.iloc[2:].copy()
    except:
        df=df[df['응답자 특성별(2)'].isin(['천안시','응답자 특성별(2)'])].T.iloc[2:].copy()

    df = df.reset_index()
    df.columns = ['연도','삶의대한만족도','비율_점']
    df['연도'] = df['연도'].str.slice(0,4)
    return df

file_list = []
for file in file_name:
    file_list.append(make_file(file))

life_satisfaction = pd.concat(file_list)
life_satisfaction.head(3)

,연도,삶의대한만족도,비율_점
0,2019,0 (전혀 만족하지 않는다) (%),0.6
1,2019,1 (%),0.8
2,2019,2 (%),2.1


#### 👉 건축허가 (입력데이터)
- 출처: 천안시
- 대상범위: 충남(천안시)
- 기간: 2019~2023년

In [44]:
buliding = pd.read_csv('1. 기초자료\천안시_건축허가.csv')
buliding = buliding.T
buliding.columns = buliding.iloc[0]
buliding = buliding[['건축용도별(1)','상업용','공업용']].copy()
buliding.columns = ['구분1','구분2','상업용_동','상업용_연면적','공업용_동','공업용_연면적']
buliding = buliding.iloc[2:]
buliding = buliding[(buliding['구분2'] =='계')&(buliding['구분1'] =='합계')].copy()
buliding = buliding.reset_index(names='연도')
buliding = buliding[['연도','상업용_동','상업용_연면적','공업용_동','공업용_연면적']]
buliding.head(2)

,연도,상업용_동,상업용_연면적,공업용_동,공업용_연면적
0,2018,1057,780450,362,310892
1,2019,762,170885,274,236032


#### 👉 기상개황 (입력데이터)
- 출처: 천안시
- 대상범위: 충남(천안시)
- 기간: 2019~2023년

In [75]:
wealther = pd.read_csv('1. 기초자료\천안시_기상개황.csv')
wealther=wealther.T
wealther.columns = wealther.iloc[0]
wealther = wealther.iloc[1:]
wealther.columns = ['구분1', '구분2', '합계', '1월', '2월', '3월', '4월', '5월', '6월', '7월', '8월','9월', '10월', '11월', '12월']
wealther =  wealther[wealther['구분1'].isin(['기온 (℃)', '강수량 (㎜)', '상대습도 (%)', '평균운량 (10%)', '일조시간 (h)', '바람 (㎧)'])].copy()
wealther.head(2)

,구분1,구분2,합계,1월,2월,3월,4월,5월,6월,7월,8월,9월,10월,11월,12월
2018,기온 (℃),평균,11.8,-4.2,-2.0,7.4,12.4,17.5,21.9,26.3,27.2,19.3,11.1,6.2,-1.2
2018.1,기온 (℃),평균최고,18.0,1.0,4.2,14.4,19.5,23.6,28.1,31.6,32.5,24.8,18.0,13.3,4.6


#### 👉 산업체 사업체 수 및 종사자 수
- 출처: 천안시
- 대상범위: 충남(천안시)
- 기간: 2019~2023년

#### 👉 소비자물가지수
- 출처: 천안시
- 대상범위: 충남(천안시)
- 기간: 2019~2023년

In [78]:
price = pd.read_csv('1. 기초자료\천안시_소비자물가지수.csv')
price.head(5)

,소비자물가별(1),2018,2019,2020,2021,2022,2023
0,총지수,99.40,99.47,100.0,102.68,108.62,112.17
1,식료품· 비주류음료,94.59,95.13,100.0,105.45,112.35,118.81
2,주류· 담배,98.77,99.67,100.0,100.34,102.75,103.63
3,의류·신발,99.76,99.76,100.0,100.55,103.64,110.70
4,"주택, 수도, 전기 및 연료",98.50,100.02,100.0,102.07,110.38,117.40


### 2. 종속 데이터 로드

In [101]:
# 지방 관서 코드 7210: 천안

a=pd.read_csv(r'1. 기초자료\산업재해현황_MDIS_재해현황\2022_재해현황_20250828_04050.csv',encoding='cp949',low_memory=False)
d=a[a['지방관서'] == 7210].copy()
d[(d['기준년'] == 2022)&(d['대업종'] == 1)]

,기준년,대업종,규모,성별,연령,근속기간,요양기간,발생형태,업무상질병1,업무상질병2,재해자구분,지방관서,지역,공사규모
681,2022,1,11,1,16,24,6,06,##,##,11,7210,11,J
799,2022,1,11,1,16,93,99,61,01,01,32,7210,11,J
800,2022,1,11,1,16,93,99,61,01,01,32,7210,11,J
804,2022,1,11,1,16,93,99,61,09,22,32,7210,11,J
907,2022,1,13,1,9,24,99,04,##,##,31,7210,11,J
919,2022,1,13,1,16,24,5,02,##,##,11,7210,11,J
953,2022,1,13,1,16,80,6,02,##,##,11,7210,11,J
1293,2022,1,22,1,16,93,1,61,01,01,12,7210,11,J
1294,2022,1,22,1,16,93,1,61,01,01,12,7210,11,J


In [108]:
import requests
import json

# 서비스키 (공공데이터포털에서 발급받은 디코딩된 키)
serviceKey = "DUEhiKmCI07pl4U62gmvzCDlQgwfyVwCVRt2jqJbVZIimwoJcxBPjC/Fu91WNlXVyU5f5R7JmLndYNYcPqxs2Q=="

# API 엔드포인트
url = "https://apis.data.go.kr/B190001/localGiftsOperateV2/operationsV2"

# 요청 파라미터 (예시: 특정 연도, 특정 지역코드)
params = {
    "serviceKey": serviceKey,  # 인증키
    "pageNo": 1,               # 페이지 번호
    "numOfRows": 10,           # 한 번에 불러올 데이터 수
}

# API 호출
response = requests.get(url, params=params)


SSLError: HTTPSConnectionPool(host='apis.data.go.kr', port=443): Max retries exceeded with url: /B190001/localGiftsOperateV2/operationsV2?serviceKey=DUEhiKmCI07pl4U62gmvzCDlQgwfyVwCVRt2jqJbVZIimwoJcxBPjC%2FFu91WNlXVyU5f5R7JmLndYNYcPqxs2Q%3D%3D&pageNo=1&numOfRows=10 (Caused by SSLError(SSLError(1, '[SSL: SSLV3_ALERT_ILLEGAL_PARAMETER] ssl/tls alert illegal parameter (_ssl.c:1000)')))